In [1]:
import tensorflow as tf
import os
import random
import pylab
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image

In [9]:
DATA_FOLDER = '/home/ankdesh/explore/DeepLearning-UdacityCapston/data/train_sample'
TFRECORDFILENAME = 'train_sample'

In [17]:
IMG_SIZE = 100 # Side for each transformed Image
IMG_DEPTH = 3 # RGB files

In [24]:
def getNumPngFiles(dirName):
    return len(glob.glob(dirName + '/*'))
def getNextImage(dirName):
    allFileNames = glob.glob(dirName + '/*')
    random.shuffle(allFileNames)
    for imgFile in allFileNames:
        img = Image.open(imgFile)
        img = img.resize((IMG_SIZE,IMG_SIZE), resample = (Image.BILINEAR))
        #my_img = tf.image.decode_png(imgFile)
        yield np.asarray(img)

In [36]:
asd = getNextImage(DATA_FOLDER).next()
asd.shape

(100, 100, 3)

In [26]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [31]:
def writeRecordFile(fileName):
  fileName = os.path.join(fileName + '.tfrecords')
  print('Writing to ...', fileName)
  writer = tf.python_io.TFRecordWriter(fileName)
  for i in range(getNumPngFiles(DATA_FOLDER)):
    image_raw = getNextImage(DATA_FOLDER).next().tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(IMG_SIZE),
        'width': _int64_feature(IMG_SIZE),
        'depth': _int64_feature(IMG_DEPTH),
        #'label': _int64_feature(int(labels[index])),
        'image_raw': _bytes_feature(image_raw)}))
    writer.write(example.SerializeToString())

In [32]:
writeRecordFile(TFRECORDFILENAME)

('Writing to ...', 'train_sample.tfrecords')
